<a href="https://colab.research.google.com/github/PBICF/PH7/blob/master/xLAM_1b_fc_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers json torch

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Salesforce/xLAM-1b-fc-r")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "\nI am an AI Programmed by IBM. I can assist with questions and tasks related to computer science, programming, and technology. I don't have personal identities, so I can't identify you as an individual."}]}]

In [3]:
# Please use our provided instruction prompt for best performance
task_instruction = """
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out and refuse to answer.
If the given question lacks the parameters required by the function, also point it out.
""".strip()

format_instruction = """
The output MUST strictly adhere to the following JSON format, and NO other text MUST be included.
The example format is as follows. Please make sure the parameter type is correct. If no function call is needed, please make tool_calls an empty list '[]'.
```
{
    "tool_calls": [
    {"name": "func_name1", "arguments": {"argument1": "value1", "argument2": "value2"}},
    ... (more tool calls as required)
    ]
}
```
""".strip()

# Define the input query and available tools
query = "What's the weather like in chennai in clcius?"

get_weather_api = {
    "name": "get_weather",
    "description": "Get the current weather for a location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, New York"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "The unit of temperature to return"
            }
        },
        "required": ["location"]
    }
}

search_api = {
    "name": "search",
    "description": "Search for information on the internet",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query, e.g. 'latest news on AI'"
            }
        },
        "required": ["query"]
    }
}

openai_format_tools = [get_weather_api, search_api]

In [4]:
import json
# Helper function to convert openai format tools to our more concise xLAM format
def convert_to_xlam_tool(tools):
    ''''''
    if isinstance(tools, dict):
        return {
            "name": tools["name"],
            "description": tools["description"],
            "parameters": {k: v for k, v in tools["parameters"].get("properties", {}).items()}
        }
    elif isinstance(tools, list):
        return [convert_to_xlam_tool(tool) for tool in tools]
    else:
        return tools

# Helper function to build the input prompt for our model
def build_prompt(task_instruction: str, format_instruction: str, tools: list, query: str):
    prompt = f"[BEGIN OF TASK INSTRUCTION]\n{task_instruction}\n[END OF TASK INSTRUCTION]\n\n"
    prompt += f"[BEGIN OF AVAILABLE TOOLS]\n{json.dumps(xlam_format_tools)}\n[END OF AVAILABLE TOOLS]\n\n"
    prompt += f"[BEGIN OF FORMAT INSTRUCTION]\n{format_instruction}\n[END OF FORMAT INSTRUCTION]\n\n"
    prompt += f"[BEGIN OF QUERY]\n{query}\n[END OF QUERY]\n\n"
    return prompt

In [5]:
# Build the input and start the inference
xlam_format_tools = convert_to_xlam_tool(openai_format_tools)
content = build_prompt(task_instruction, format_instruction, xlam_format_tools, query)

In [6]:
def get_weather(location: str, unit: str = "celsius") -> str:
    """Simulates fetching current weather for a given location and unit."""
    print(f"Simulating weather API call for {location} in {unit}.")

    # In a real application, you would make an API call here, e.g.:
    import requests
    api_key = "98a0d31c429c49a8ae541644251112"
    url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={location}"
    response = requests.get(url).json()

    # Process response to extract temperature in the desired unit
    if unit == "fahrenheit":
        temperature = response["current"]["temp_f"]
    else: # default to celsius
        temperature = response["current"]["temp_c"]
    return f"The current temperature in {location} is {temperature}°{unit[0].upper()}."

In [7]:
import json

def handle_tool_calls(model_output):
    try:
        # Assuming model_output is a string like: {"tool_calls": [...]}
        parsed_output = json.loads(model_output)
        if "tool_calls" in parsed_output and isinstance(parsed_output["tool_calls"], list):
            for tool_call in parsed_output["tool_calls"]:
                tool_name = tool_call.get("name")
                tool_args = tool_call.get("arguments", {})

                if tool_name == "get_weather":
                    print(f"Calling get_weather with arguments: {tool_args}")
                    # Call the actual get_weather function
                    result = get_weather(location=tool_args.get("location"), unit=tool_args.get("unit"))
                    print(f"Weather result: {result}")
                elif tool_name == "search":
                    print(f"Calling search with arguments: {tool_args}")
                    # Implement search function call here
                    print("Search function is not implemented yet.")
                else:
                    print(f"Unknown tool: {tool_name}")
        else:
            print("No valid 'tool_calls' found in the model output.")
    except json.JSONDecodeError:
        print("Invalid JSON output from model.")
    except Exception as e:
        print(f"An error occurred while handling tool calls: {e}")


In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/xLAM-1b-fc-r")
model = AutoModelForCausalLM.from_pretrained("Salesforce/xLAM-1b-fc-r")

messages=[
    { 'role': 'user', 'content': content}
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
decoded_output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(decoded_output)
handle_tool_calls(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



{"tool_calls": [{"name": "get_weather", "arguments": {"location": "chennai", "unit": "celsius"}}]}
Calling get_weather with arguments: {'location': 'chennai', 'unit': 'celsius'}
Simulating weather API call for chennai in celsius.
Weather result: The current temperature in chennai is 27.2°C.
